## Scraping PC Gaming
L'objectif de ce notebook est de récupérer toutes les informations disponibles sur tous les PC Gaming listés sur le site Boulanger.

Lien de la page des PC Gaming https://www.boulanger.com/c/pc-gamer?

## Imports et initialisation des variables à récupérer

In [1]:
import requests
import time
import pandas as pd
###60 sec=1 min
from bs4 import BeautifulSoup

#Initialisation des variables a récupérer 
ls_product_name = [] #stocke le nom des pc
ls_characteristics = [] #stocke les caractéristiques des pc
ls_services = [] #stocke les services offerts pour les pc
ls_price = [] #stocke le prix des pc


## Récupération des infos d'un pc : Fonctionnel
Points à améliorer:
- Trouver comment convertir le contenu characteristics en texte sans passer par l'usage de text_characteristics.
- Trouver comment afficher le contenu des containers de la rubrique services. Pour ça faut utiliser [Selenium](https://selenium-python.readthedocs.io) 

In [112]:
url = "https://www.boulanger.com/ref/8006657"

response = requests.get(url)
response_html = BeautifulSoup(response.text, 'html.parser')

title = response_html.find('h1').text  #ok                                      #product-title__main
#print(title)
price = response_html.find('p', class_="price__amount").text   #ok
#print(price)

characteristics = response_html.find('section', class_="keypoints keypoints--with-background").find_all('li')#ok mais j'arrive pas à convertir en texte parce que find_all retourne une liste

text_characteristics = str(characteristics)                                     #donc voila mon work around : je convertis donc la liste characteristics en str 



text_to_delete = {'</li>, <li class="keypoints__item">':'',                     #je cree une liste key:value des mots que je veux remplacer(key) par le vide (value) de mon str
                  '</li>, <li class="keypoints__item keypoints__item--advantage">':'',
                  '<strong>':'', '</strong>':'', '</li>':'',']':'','-':'','[<li class="keypoints__item">':''}


for key, value in text_to_delete.items():                                       #je parcours ma liste
    text_characteristics = text_characteristics.replace(key, value)             #je remplace les mots a supprimer(key) par le vide (value) dans mon string

#print(text_characteristics)                                                     #text_characteristics sera la variable a utliser pour parcourir tout le site


services = response_html.find('div', id="services").text                        #ok pour le moment 
#services = response_html.find('div', class_="product-services__all").text
#services = response_html.find('div', id='container-warranty')                  #J'n'arrive pas à afficher le contenu des containers de la rubrique service jsp pk 
print(services)

  




    Prolongez votre garantie



    Faites-vous accompagner par Boulanger



    Souscrivez à une assurance






## Récupération de tous les pc gaming : Fonctionnel
Points à améliorer:
- Trouver comment afficher le contenu des containers de la rubrique services.

In [118]:
''' Observations
Étant donné qu'il y a 915 articles disponibles sur le site, 
il est fort probable que je ne sauvegarde pas toutes leurs données dans mon df final.
Cependant, des tests ont été effectués sur 11 pages et je peux attester à 100% que mon code est fonctionnel. 
'''
page = 1
while True:
  print(page)

  url ='https://www.boulanger.com/c/pc-gamer?numPage='+ str(page)

  response = requests.get(url)

  if response.status_code != 200:
    break

  soup = BeautifulSoup(response.text, 'html.parser')

  pc_links = soup.find_all('a', class_='product-item__image-link analytic-track-origin') #stock les 30 balises <a> contenant les liens vers les 30 pc de la page

  for link in pc_links:

    new_url = 'https://www.boulanger.com/' + link['href']  
    new_response = requests.get(new_url)
    new_response_html = BeautifulSoup(new_response.text, 'html.parser')
    #print(new_url)

    title = new_response_html.find('h1').text
    #print(title)

    price = new_response_html.find('p', class_="price__amount").text 
    #print(price)

    services = new_response_html.find('div', id="services").text #ici ne retourne que des "None" et le .text ne marche plus
    #print(services)

    characteristics = new_response_html.find('section', class_="keypoints keypoints--with-background").find_all('li')
    text_characteristics = str(characteristics)

    #je cree une liste key:value des mots que je veux remplacer(key) par le vide (value) de mon str
    text_to_delete = {'</li>, <li class="keypoints__item">':'',                     
                      '</li>, <li class="keypoints__item keypoints__item--advantage">':'',
                      '<strong>':'', '</strong>':'', '</li>':'',']':'','-':'','[<li class="keypoints__item">':''}

    #je parcours ma liste et remplace les mots a supprimer(key) par le vide (value) dans mon string
    for key, value in text_to_delete.items():                                       
        text_characteristics = text_characteristics.replace(key, value)
    #print(text_characteristics)                                                #text_characteristics sera la variable a append a ls_characteristics

    ls_product_name.append(title)
    ls_price.append(price)
    ls_characteristics.append(text_characteristics)
    ls_services.append(services)
  
  page += 1

1
2
3
4
5
6
7
8
9
10
11


KeyboardInterrupt: ignored

## To CSV

In [121]:
pd.reset_option('max_columns')
df = pd.DataFrame({'PC NAME':ls_product_name, 'CHARACTERISTICS':ls_characteristics, 'SERVICES':ls_services, 'PRICE':ls_price})

In [122]:
df.head()

,PC NAME,CHARACTERISTICS,SERVICES,PRICE
0,\n\t\t\t\t\n\t\n\t\tPC Gamer\n\t\t\n\t\t\n\t\t...,"[<li class=""keypoints__item"">\n ...",\n\n\n Prolongez votre garantie\n\n\n\n ...,"\n 1699,99€\n"
1,\n\t\t\t\t\n\t\n\t\tPC Gamer\n\t\t\n\t\t\n\t\t...,"[<li class=""keypoints__item"">\n ...",\n\n\n Prolongez votre garantie\n\n\n\n ...,"\n 1699,99€\n"
2,\n\t\t\t\t\n\t\n\t\tPC Gamer\n\t\t\n\t\t\n\t\t...,"[<li class=""keypoints__item"">\n ...",\n\n\n Prolongez votre garantie\n\n\n\n ...,"\n 1699,99€\n"
3,\n\t\t\t\t\n\t\n\t\tPC Gamer\n\t\t\n\t\t\n\t\t...,"[<li class=""keypoints__item"">\n ...",\n\n\n Prolongez votre garantie\n\n\n\n ...,"\n 1699,99€\n"
4,\n\t\t\t\t\n\t\n\t\tPC Gamer\n\t\t\n\t\t\n\t\t...,"[<li class=""keypoints__item"">\n ...",\n\n\n Prolongez votre garantie\n\n\n\n ...,"\n 1699,99€\n"


In [123]:
df.to_csv(r'\PC_Gaming_Infos.csv')

## Draft

In [ ]:
#https://www.boulanger.com/c/pc-gamer?numPage=1
while True:

  url = 'https://www.boulanger.com/c/pc-gamer?numPage='+ str(page)
  
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  
  pc_links = soup.find('article', class_="col-24 product-item").div["col-16"]
  
  
  time.sleep(60)


  if pc_links == []:
    print('End')
    break


  else:
    for link in pc_links:
      all_pc_links.append("https://www.boulanger.com/c/pc-gamer"+str(link.find('a')['href']))     
    page += 1


#Boucler sur les 874 liens des pages de chaque pc pour en récuperer les details 
for all_links in all_pc_links:
  responseDuLivre = requests.get(pc_links)
  pc_soup = BeautifulSoup(responseDuLivre.text, 'html.parser')


  #Recuperation du nom du produit
  title = pc_soup.find('div', class_='product-title col--22')
  ls_product_name.append(title.find("h1" , recursive=False).text) #permet de recuperer le texte de la premiere balise enfant


  #Recuperation des caractéristiques 
  characteristics = pc_soup.find('section', class_="keypoints keypoints--with-background")
  #ls_characteristics.append(characteristics.find().text)


  #Recuperation des services
  services = pc_soup.find('div', class_='col-sm-6 product_main')
  #ls_services.append(services.find('div', class_='anchors')).find('li', class_='anchors_item').find('a')['href'].text)
  services = services.find('div', class_='anchors').find('li', class_='anchors_item').find('a')['href']
  services

  #Recuperation du prix
  price = pc_soup.find('div', class_="product-sidebar col-9")
  ls_price.append(price.find('div', class_="price ").text)

## Tests
Ici je bidouille mes codes safely pour ne pas corrompre ce qui marche deja.
Et je daily blog mes problèmes et soluces.

- Ma boucle tournait sur le meme lien car je passais a new_url, 'https://www.boulanger.com/c/pc-gamer?'+ link[href]. Cependant, comme ces liens n'existaient pas, le site me renvoyait par defaut a la page 1 des pc gaming, 30 fois.
- Pour régler ça, j'ai du passer le lien de la page d'accueil de Boulanger + link[href]

In [101]:
url ='https://www.boulanger.com/c/pc-gamer?numPage='+ str(page)

response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

pc_links = soup.find_all('a', class_='product-item__image-link analytic-track-origin') #stock les 30 balises <a> contenant les liens vers les 30 pc de la page

for link in pc_links:

  new_url = 'https://www.boulanger.com/' + link['href']  
  new_response = requests.get(new_url)
  new_response_html = BeautifulSoup(new_response.text, 'html.parser')
  #print(new_url)
  title = response_html.find('h1').text
  print(title)
  price = response_html.find('p', class_="price__amount").text 

  characteristics = response_html.find('section', class_="keypoints keypoints--with-background").find_all('li')
  text_characteristics = str(characteristics)                                  
  #je cree une liste key:value des mots que je veux remplacer(key) par le vide (value) de mon str
  text_to_delete = {'</li>, <li class="keypoints__item">':'',                     
                    '</li>, <li class="keypoints__item keypoints__item--advantage">':'',
                    '<strong>':'', '</strong>':'', '</li>':'',']':'','-':''}
  #je parcours ma liste et remplace les mots a supprimer(key) par le vide (value) dans mon string
  for key, value in text_to_delete.items():                                       
      text_characteristics = text_characteristics.replace(key, value)
  
  #text_characteristics sera la variable a append a ls_characteristics
  #print(text_characteristics)


				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		
		
			SKILLKORP
		
		SKP H6
	
	
	

			

				
	
		Casque gamer
		